In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
from tqdm import tqdm
import json
import joblib
import numpy as np
import os

from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold, KFold
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer, TrainerCallback
import evaluate
import torch

tqdm.pandas()

# Interaction data

In [9]:
df_interaction = pd.read_csv("interaction_data.csv")
df_interaction

,correctTag,targetWord,eventType,orderingId,sessionId,draggedTag
0,Verbo,Será,interaction,0,xqEk5ZfKaG,Pronome
1,Verbo,Será,interaction,1,xqEk5ZfKaG,Conjunção Subordinativa
2,Pontuação,",",interaction,0,uQY36KP907,Pontuação
3,Pontuação,!,interaction,1,uQY36KP907,Pontuação
4,Substantivo,árvore,interaction,0,X2RK9EgWWZ,Substantivo
...,...,...,...,...,...,...
14179,Pronome,Você,interaction,1,oYoPEopG2Z,Pronome
14180,Verbo,pensou,interaction,2,oYoPEopG2Z,Verbo
14181,Verbo,fazer,interaction,3,oYoPEopG2Z,Verbo
14182,Preposição+Artigo,nas,interaction,4,oYoPEopG2Z,Preposição+Artigo


In [10]:
# Cria coluna indicando se houve erro de etiquetagem por parte do aluno
def check_for_mistakes(row):
    return row['correctTag'] != row['draggedTag']

df_interaction['isMistake'] = df_interaction.apply(check_for_mistakes, axis=1)

In [11]:
df_interaction

,correctTag,targetWord,eventType,orderingId,sessionId,draggedTag,isMistake
0,Verbo,Será,interaction,0,xqEk5ZfKaG,Pronome,True
1,Verbo,Será,interaction,1,xqEk5ZfKaG,Conjunção Subordinativa,True
2,Pontuação,",",interaction,0,uQY36KP907,Pontuação,False
3,Pontuação,!,interaction,1,uQY36KP907,Pontuação,False
4,Substantivo,árvore,interaction,0,X2RK9EgWWZ,Substantivo,False
...,...,...,...,...,...,...,...
14179,Pronome,Você,interaction,1,oYoPEopG2Z,Pronome,False
14180,Verbo,pensou,interaction,2,oYoPEopG2Z,Verbo,False
14181,Verbo,fazer,interaction,3,oYoPEopG2Z,Verbo,False
14182,Preposição+Artigo,nas,interaction,4,oYoPEopG2Z,Preposição+Artigo,False


In [12]:
df_interaction.isMistake.value_counts()

True     7479
False    6705
Name: isMistake, dtype: int64

In [13]:
df_interaction.isMistake.value_counts(True)

True     0.527284
False    0.472716
Name: isMistake, dtype: float64

In [14]:
df_interaction.to_pickle("interaction_data.pkl")

# Session data

In [18]:
df_session = pd.read_csv("session_data.csv")
df_session

,startTime,serie,words,eventType,sessionId
0,2025-05-27 00:49:01.547000+00:00,5-ensino-fundamental,"[{'tag': 'Verbo', 'word': 'Será'}, {'tag': 'Co...",new-session,xqEk5ZfKaG
1,2025-05-27 01:01:42.509000+00:00,5-ensino-fundamental,"[{'tag': 'Nome Próprio', 'word': 'Ufa'}, {'tag...",new-session,uQY36KP907
2,2025-05-27 11:11:54.325000+00:00,6-ensino-fundamental,"[{'tag': 'Artigo', 'word': 'A'}, {'tag': 'Subs...",new-session,X2RK9EgWWZ
3,2025-05-27 11:12:37.255000+00:00,2-ensino-medio,"[{'tag': 'Pronome', 'word': 'Você'}, {'tag': '...",new-session,IONQQWNtUo
4,2025-05-27 12:21:30.712000+00:00,1-ensino-medio,"[{'tag': 'Advérbio', 'word': 'Depois'}, {'tag'...",new-session,brmbB7P32D
...,...,...,...,...,...
1276,2025-05-30 13:53:20.637000+00:00,8-ensino-fundamental,"[{'tag': 'Artigo', 'word': 'A'}, {'tag': 'Subs...",new-session,lRItfgU8dO
1277,2025-05-30 13:54:22.309000+00:00,8-ensino-fundamental,"[{'tag': 'Artigo', 'word': 'O'}, {'tag': 'Subs...",new-session,gNa9YP3fj1
1278,2025-05-30 14:11:43.647000+00:00,8-ensino-fundamental,"[{'tag': 'Artigo', 'word': 'A'}, {'tag': 'Subs...",new-session,CMTDvNvzsh
1279,2025-05-30 16:15:09.209000+00:00,9-ensino-fundamental,"[{'tag': 'Conjunção Subordinativa', 'word': 'A...",new-session,wYJqZLwSJd


In [19]:
# Criando coluna com as frases originais em si
df_session['words'] = df_session['words'].progress_apply(lambda x: json.loads(x.replace("'", '"')))
df_session['sentence'] = df_session['words'].progress_apply(lambda x: ' '.join([token_info['word'] for token_info in x]))

  0%|          | 0/1281 [00:00<?, ?it/s]

  0%|          | 0/1281 [00:00<?, ?it/s]

In [20]:
df_session

,startTime,serie,words,eventType,sessionId,sentence
0,2025-05-27 00:49:01.547000+00:00,5-ensino-fundamental,"[{'tag': 'Verbo', 'word': 'Será'}, {'tag': 'Co...",new-session,xqEk5ZfKaG,Será que alguém explicou o que aconteceu na au...
1,2025-05-27 01:01:42.509000+00:00,5-ensino-fundamental,"[{'tag': 'Nome Próprio', 'word': 'Ufa'}, {'tag...",new-session,uQY36KP907,"Ufa , achei que não fosse chegar a tempo !"
2,2025-05-27 11:11:54.325000+00:00,6-ensino-fundamental,"[{'tag': 'Artigo', 'word': 'A'}, {'tag': 'Subs...",new-session,X2RK9EgWWZ,A folha caiu da árvore .
3,2025-05-27 11:12:37.255000+00:00,2-ensino-medio,"[{'tag': 'Pronome', 'word': 'Você'}, {'tag': '...",new-session,IONQQWNtUo,Você já terminou o dever de casa ?
4,2025-05-27 12:21:30.712000+00:00,1-ensino-medio,"[{'tag': 'Advérbio', 'word': 'Depois'}, {'tag'...",new-session,brmbB7P32D,"Depois que o sol se pôs , o frio começou a aum..."
...,...,...,...,...,...,...
1276,2025-05-30 13:53:20.637000+00:00,8-ensino-fundamental,"[{'tag': 'Artigo', 'word': 'A'}, {'tag': 'Subs...",new-session,lRItfgU8dO,A menina caiu na escada e machucou o braço .
1277,2025-05-30 13:54:22.309000+00:00,8-ensino-fundamental,"[{'tag': 'Artigo', 'word': 'O'}, {'tag': 'Subs...",new-session,gNa9YP3fj1,O aluno nota 10 tirou uma nota ruim .
1278,2025-05-30 14:11:43.647000+00:00,8-ensino-fundamental,"[{'tag': 'Artigo', 'word': 'A'}, {'tag': 'Subs...",new-session,CMTDvNvzsh,A moça que canta chegou agora .
1279,2025-05-30 16:15:09.209000+00:00,9-ensino-fundamental,"[{'tag': 'Conjunção Subordinativa', 'word': 'A...",new-session,wYJqZLwSJd,"Assim que terminou a reunião , ela ligou para ..."


In [21]:
df_session.to_pickle('session_data.pkl')

# Merging dataframes

In [22]:
df_session = pd.read_pickle("session_data.pkl")
df_interaction = pd.read_pickle("interaction_data.pkl")

In [23]:
df_session

,startTime,serie,words,eventType,sessionId,sentence
0,2025-05-27 00:49:01.547000+00:00,5-ensino-fundamental,"[{'tag': 'Verbo', 'word': 'Será'}, {'tag': 'Co...",new-session,xqEk5ZfKaG,Será que alguém explicou o que aconteceu na au...
1,2025-05-27 01:01:42.509000+00:00,5-ensino-fundamental,"[{'tag': 'Nome Próprio', 'word': 'Ufa'}, {'tag...",new-session,uQY36KP907,"Ufa , achei que não fosse chegar a tempo !"
2,2025-05-27 11:11:54.325000+00:00,6-ensino-fundamental,"[{'tag': 'Artigo', 'word': 'A'}, {'tag': 'Subs...",new-session,X2RK9EgWWZ,A folha caiu da árvore .
3,2025-05-27 11:12:37.255000+00:00,2-ensino-medio,"[{'tag': 'Pronome', 'word': 'Você'}, {'tag': '...",new-session,IONQQWNtUo,Você já terminou o dever de casa ?
4,2025-05-27 12:21:30.712000+00:00,1-ensino-medio,"[{'tag': 'Advérbio', 'word': 'Depois'}, {'tag'...",new-session,brmbB7P32D,"Depois que o sol se pôs , o frio começou a aum..."
...,...,...,...,...,...,...
1276,2025-05-30 13:53:20.637000+00:00,8-ensino-fundamental,"[{'tag': 'Artigo', 'word': 'A'}, {'tag': 'Subs...",new-session,lRItfgU8dO,A menina caiu na escada e machucou o braço .
1277,2025-05-30 13:54:22.309000+00:00,8-ensino-fundamental,"[{'tag': 'Artigo', 'word': 'O'}, {'tag': 'Subs...",new-session,gNa9YP3fj1,O aluno nota 10 tirou uma nota ruim .
1278,2025-05-30 14:11:43.647000+00:00,8-ensino-fundamental,"[{'tag': 'Artigo', 'word': 'A'}, {'tag': 'Subs...",new-session,CMTDvNvzsh,A moça que canta chegou agora .
1279,2025-05-30 16:15:09.209000+00:00,9-ensino-fundamental,"[{'tag': 'Conjunção Subordinativa', 'word': 'A...",new-session,wYJqZLwSJd,"Assim que terminou a reunião , ela ligou para ..."


In [24]:
df_interaction

,correctTag,targetWord,eventType,orderingId,sessionId,draggedTag,isMistake
0,Verbo,Será,interaction,0,xqEk5ZfKaG,Pronome,True
1,Verbo,Será,interaction,1,xqEk5ZfKaG,Conjunção Subordinativa,True
2,Pontuação,",",interaction,0,uQY36KP907,Pontuação,False
3,Pontuação,!,interaction,1,uQY36KP907,Pontuação,False
4,Substantivo,árvore,interaction,0,X2RK9EgWWZ,Substantivo,False
...,...,...,...,...,...,...,...
14179,Pronome,Você,interaction,1,oYoPEopG2Z,Pronome,False
14180,Verbo,pensou,interaction,2,oYoPEopG2Z,Verbo,False
14181,Verbo,fazer,interaction,3,oYoPEopG2Z,Verbo,False
14182,Preposição+Artigo,nas,interaction,4,oYoPEopG2Z,Preposição+Artigo,False


In [25]:
# Join dos datasets em sessionId, mantendo as chaves da tabela de interações (maior)
df = pd.merge(df_interaction, df_session, how="left", left_on="sessionId", right_on="sessionId")
df

,correctTag,targetWord,eventType_x,orderingId,sessionId,draggedTag,isMistake,startTime,serie,words,eventType_y,sentence
0,Verbo,Será,interaction,0,xqEk5ZfKaG,Pronome,True,2025-05-27 00:49:01.547000+00:00,5-ensino-fundamental,"[{'tag': 'Verbo', 'word': 'Será'}, {'tag': 'Co...",new-session,Será que alguém explicou o que aconteceu na au...
1,Verbo,Será,interaction,1,xqEk5ZfKaG,Conjunção Subordinativa,True,2025-05-27 00:49:01.547000+00:00,5-ensino-fundamental,"[{'tag': 'Verbo', 'word': 'Será'}, {'tag': 'Co...",new-session,Será que alguém explicou o que aconteceu na au...
2,Pontuação,",",interaction,0,uQY36KP907,Pontuação,False,2025-05-27 01:01:42.509000+00:00,5-ensino-fundamental,"[{'tag': 'Nome Próprio', 'word': 'Ufa'}, {'tag...",new-session,"Ufa , achei que não fosse chegar a tempo !"
3,Pontuação,!,interaction,1,uQY36KP907,Pontuação,False,2025-05-27 01:01:42.509000+00:00,5-ensino-fundamental,"[{'tag': 'Nome Próprio', 'word': 'Ufa'}, {'tag...",new-session,"Ufa , achei que não fosse chegar a tempo !"
4,Substantivo,árvore,interaction,0,X2RK9EgWWZ,Substantivo,False,2025-05-27 11:11:54.325000+00:00,6-ensino-fundamental,"[{'tag': 'Artigo', 'word': 'A'}, {'tag': 'Subs...",new-session,A folha caiu da árvore .
...,...,...,...,...,...,...,...,...,...,...,...,...
14179,Pronome,Você,interaction,1,oYoPEopG2Z,Pronome,False,2025-05-30 16:52:53.776000+00:00,5-ensino-fundamental,"[{'tag': 'Pronome', 'word': 'Você'}, {'tag': '...",new-session,Você já pensou no que vai fazer nas férias ?
14180,Verbo,pensou,interaction,2,oYoPEopG2Z,Verbo,False,2025-05-30 16:52:53.776000+00:00,5-ensino-fundamental,"[{'tag': 'Pronome', 'word': 'Você'}, {'tag': '...",new-session,Você já pensou no que vai fazer nas férias ?
14181,Verbo,fazer,interaction,3,oYoPEopG2Z,Verbo,False,2025-05-30 16:52:53.776000+00:00,5-ensino-fundamental,"[{'tag': 'Pronome', 'word': 'Você'}, {'tag': '...",new-session,Você já pensou no que vai fazer nas férias ?
14182,Preposição+Artigo,nas,interaction,4,oYoPEopG2Z,Preposição+Artigo,False,2025-05-30 16:52:53.776000+00:00,5-ensino-fundamental,"[{'tag': 'Pronome', 'word': 'Você'}, {'tag': '...",new-session,Você já pensou no que vai fazer nas férias ?


In [26]:
df_unique_sessions = df.groupby('sessionId').last()
df_unique_sessions

,correctTag,targetWord,eventType_x,orderingId,draggedTag,isMistake,startTime,serie,words,eventType_y,sentence
sessionId,,,,,,,,,,,
-6gZ-mHdzv,Substantivo,ideia,interaction,7,Substantivo,False,2025-05-29 01:28:34.290000+00:00,9-ensino-fundamental,"[{'tag': 'Artigo', 'word': 'O'}, {'tag': 'Subs...",new-session,O diretor aprovou a ideia dela .
-CS8WhEL5b,Conjunção Subordinativa,Embora,interaction,14,Conjunção Subordinativa,False,2025-05-29 21:48:03.660000+00:00,3-ensino-medio,"[{'tag': 'Conjunção Subordinativa', 'word': 'E...",new-session,"Embora a chuva tenha parado , o chão ainda est..."
-FzZZhJbxP,Verbo,abertos,interaction,18,Verbo,False,2025-05-28 22:17:08.503000+00:00,3-ensino-medio,"[{'tag': 'Pronome', 'word': 'Ela'}, {'tag': 'V...",new-session,Ela estava tão cansada que mal conseguia mante...
-GYoamgajp,Conjunção Subordinativa,Se,interaction,8,Conjunção Subordinativa,False,2025-05-28 19:48:11.714000+00:00,9-ensino-fundamental,"[{'tag': 'Conjunção Subordinativa', 'word': 'S...",new-session,"Se chover , vamos ficar em casa ."
-K7PSmnAj6,Conjunção Subordinativa,que,interaction,21,Conjunção Subordinativa,False,2025-05-30 00:49:30.724000+00:00,9-ensino-fundamental,"[{'tag': 'Interjeição', 'word': 'Meu'}, {'tag'...",new-session,"Meu caro João , espero que esteja se sentindo ..."
...,...,...,...,...,...,...,...,...,...,...,...
zctkxMLR1j,Substantivo,parque,interaction,13,Substantivo,False,2025-05-29 14:34:37.748000+00:00,9-ensino-fundamental,"[{'tag': 'Pronome', 'word': 'Ela'}, {'tag': 'V...",new-session,Ela perguntou se eu queria ir ao parque no dom...
zr_27p7xlg,Adjetivo,atrasado,interaction,19,Adjetivo,False,2025-05-29 16:54:35.924000+00:00,9-ensino-fundamental,"[{'tag': 'Advérbio', 'word': 'Finalmente'}, {'...",new-session,"Finalmente , conseguimos terminar o projeto qu..."
zyUGx8O6BL,Pontuação,.,interaction,11,Pontuação,False,2025-05-30 00:13:21.324000+00:00,8-ensino-fundamental,"[{'tag': 'Conjunção Subordinativa', 'word': 'E...",new-session,"Enquanto ele falava , todos prestavam muita at..."


In [27]:
def check_for_complete_tries(row):
    return len(row['words']) == row['orderingId']

df_unique_sessions['complete_try'] = df_unique_sessions.apply(check_for_complete_tries, axis=1)

In [28]:
df_unique_sessions

,correctTag,targetWord,eventType_x,orderingId,draggedTag,isMistake,startTime,serie,words,eventType_y,sentence,complete_try
sessionId,,,,,,,,,,,,
-6gZ-mHdzv,Substantivo,ideia,interaction,7,Substantivo,False,2025-05-29 01:28:34.290000+00:00,9-ensino-fundamental,"[{'tag': 'Artigo', 'word': 'O'}, {'tag': 'Subs...",new-session,O diretor aprovou a ideia dela .,True
-CS8WhEL5b,Conjunção Subordinativa,Embora,interaction,14,Conjunção Subordinativa,False,2025-05-29 21:48:03.660000+00:00,3-ensino-medio,"[{'tag': 'Conjunção Subordinativa', 'word': 'E...",new-session,"Embora a chuva tenha parado , o chão ainda est...",False
-FzZZhJbxP,Verbo,abertos,interaction,18,Verbo,False,2025-05-28 22:17:08.503000+00:00,3-ensino-medio,"[{'tag': 'Pronome', 'word': 'Ela'}, {'tag': 'V...",new-session,Ela estava tão cansada que mal conseguia mante...,False
-GYoamgajp,Conjunção Subordinativa,Se,interaction,8,Conjunção Subordinativa,False,2025-05-28 19:48:11.714000+00:00,9-ensino-fundamental,"[{'tag': 'Conjunção Subordinativa', 'word': 'S...",new-session,"Se chover , vamos ficar em casa .",True
-K7PSmnAj6,Conjunção Subordinativa,que,interaction,21,Conjunção Subordinativa,False,2025-05-30 00:49:30.724000+00:00,9-ensino-fundamental,"[{'tag': 'Interjeição', 'word': 'Meu'}, {'tag'...",new-session,"Meu caro João , espero que esteja se sentindo ...",False
...,...,...,...,...,...,...,...,...,...,...,...,...
zctkxMLR1j,Substantivo,parque,interaction,13,Substantivo,False,2025-05-29 14:34:37.748000+00:00,9-ensino-fundamental,"[{'tag': 'Pronome', 'word': 'Ela'}, {'tag': 'V...",new-session,Ela perguntou se eu queria ir ao parque no dom...,False
zr_27p7xlg,Adjetivo,atrasado,interaction,19,Adjetivo,False,2025-05-29 16:54:35.924000+00:00,9-ensino-fundamental,"[{'tag': 'Advérbio', 'word': 'Finalmente'}, {'...",new-session,"Finalmente , conseguimos terminar o projeto qu...",False
zyUGx8O6BL,Pontuação,.,interaction,11,Pontuação,False,2025-05-30 00:13:21.324000+00:00,8-ensino-fundamental,"[{'tag': 'Conjunção Subordinativa', 'word': 'E...",new-session,"Enquanto ele falava , todos prestavam muita at...",False


In [29]:
df_unique_sessions.complete_try.value_counts()

False    826
True      69
Name: complete_try, dtype: int64

In [30]:
complete_tries = []
for index, row in tqdm(df.iterrows()):
    if df_unique_sessions.loc[row['sessionId']]['complete_try']:
        complete_tries.append('Sim')
    else:
        complete_tries.append('Não')

df['complete_try'] = complete_tries

0it [00:00, ?it/s]

In [31]:
df

,correctTag,targetWord,eventType_x,orderingId,sessionId,draggedTag,isMistake,startTime,serie,words,eventType_y,sentence,complete_try
0,Verbo,Será,interaction,0,xqEk5ZfKaG,Pronome,True,2025-05-27 00:49:01.547000+00:00,5-ensino-fundamental,"[{'tag': 'Verbo', 'word': 'Será'}, {'tag': 'Co...",new-session,Será que alguém explicou o que aconteceu na au...,Não
1,Verbo,Será,interaction,1,xqEk5ZfKaG,Conjunção Subordinativa,True,2025-05-27 00:49:01.547000+00:00,5-ensino-fundamental,"[{'tag': 'Verbo', 'word': 'Será'}, {'tag': 'Co...",new-session,Será que alguém explicou o que aconteceu na au...,Não
2,Pontuação,",",interaction,0,uQY36KP907,Pontuação,False,2025-05-27 01:01:42.509000+00:00,5-ensino-fundamental,"[{'tag': 'Nome Próprio', 'word': 'Ufa'}, {'tag...",new-session,"Ufa , achei que não fosse chegar a tempo !",Não
3,Pontuação,!,interaction,1,uQY36KP907,Pontuação,False,2025-05-27 01:01:42.509000+00:00,5-ensino-fundamental,"[{'tag': 'Nome Próprio', 'word': 'Ufa'}, {'tag...",new-session,"Ufa , achei que não fosse chegar a tempo !",Não
4,Substantivo,árvore,interaction,0,X2RK9EgWWZ,Substantivo,False,2025-05-27 11:11:54.325000+00:00,6-ensino-fundamental,"[{'tag': 'Artigo', 'word': 'A'}, {'tag': 'Subs...",new-session,A folha caiu da árvore .,Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14179,Pronome,Você,interaction,1,oYoPEopG2Z,Pronome,False,2025-05-30 16:52:53.776000+00:00,5-ensino-fundamental,"[{'tag': 'Pronome', 'word': 'Você'}, {'tag': '...",new-session,Você já pensou no que vai fazer nas férias ?,Não
14180,Verbo,pensou,interaction,2,oYoPEopG2Z,Verbo,False,2025-05-30 16:52:53.776000+00:00,5-ensino-fundamental,"[{'tag': 'Pronome', 'word': 'Você'}, {'tag': '...",new-session,Você já pensou no que vai fazer nas férias ?,Não
14181,Verbo,fazer,interaction,3,oYoPEopG2Z,Verbo,False,2025-05-30 16:52:53.776000+00:00,5-ensino-fundamental,"[{'tag': 'Pronome', 'word': 'Você'}, {'tag': '...",new-session,Você já pensou no que vai fazer nas férias ?,Não
14182,Preposição+Artigo,nas,interaction,4,oYoPEopG2Z,Preposição+Artigo,False,2025-05-30 16:52:53.776000+00:00,5-ensino-fundamental,"[{'tag': 'Pronome', 'word': 'Você'}, {'tag': '...",new-session,Você já pensou no que vai fazer nas férias ?,Não


In [32]:
df.to_pickle("merged_data.pkl")

# Montando dataset
features importantes:

- targetWord
- correctTag
- draggedTag
- isMistake
- sentence
- orderingId

Target:
- serie


In [33]:
df = pd.read_pickle('merged_data.pkl')
df

,correctTag,targetWord,eventType_x,orderingId,sessionId,draggedTag,isMistake,startTime,serie,words,eventType_y,sentence,complete_try
0,Verbo,Será,interaction,0,xqEk5ZfKaG,Pronome,True,2025-05-27 00:49:01.547000+00:00,5-ensino-fundamental,"[{'tag': 'Verbo', 'word': 'Será'}, {'tag': 'Co...",new-session,Será que alguém explicou o que aconteceu na au...,Não
1,Verbo,Será,interaction,1,xqEk5ZfKaG,Conjunção Subordinativa,True,2025-05-27 00:49:01.547000+00:00,5-ensino-fundamental,"[{'tag': 'Verbo', 'word': 'Será'}, {'tag': 'Co...",new-session,Será que alguém explicou o que aconteceu na au...,Não
2,Pontuação,",",interaction,0,uQY36KP907,Pontuação,False,2025-05-27 01:01:42.509000+00:00,5-ensino-fundamental,"[{'tag': 'Nome Próprio', 'word': 'Ufa'}, {'tag...",new-session,"Ufa , achei que não fosse chegar a tempo !",Não
3,Pontuação,!,interaction,1,uQY36KP907,Pontuação,False,2025-05-27 01:01:42.509000+00:00,5-ensino-fundamental,"[{'tag': 'Nome Próprio', 'word': 'Ufa'}, {'tag...",new-session,"Ufa , achei que não fosse chegar a tempo !",Não
4,Substantivo,árvore,interaction,0,X2RK9EgWWZ,Substantivo,False,2025-05-27 11:11:54.325000+00:00,6-ensino-fundamental,"[{'tag': 'Artigo', 'word': 'A'}, {'tag': 'Subs...",new-session,A folha caiu da árvore .,Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14179,Pronome,Você,interaction,1,oYoPEopG2Z,Pronome,False,2025-05-30 16:52:53.776000+00:00,5-ensino-fundamental,"[{'tag': 'Pronome', 'word': 'Você'}, {'tag': '...",new-session,Você já pensou no que vai fazer nas férias ?,Não
14180,Verbo,pensou,interaction,2,oYoPEopG2Z,Verbo,False,2025-05-30 16:52:53.776000+00:00,5-ensino-fundamental,"[{'tag': 'Pronome', 'word': 'Você'}, {'tag': '...",new-session,Você já pensou no que vai fazer nas férias ?,Não
14181,Verbo,fazer,interaction,3,oYoPEopG2Z,Verbo,False,2025-05-30 16:52:53.776000+00:00,5-ensino-fundamental,"[{'tag': 'Pronome', 'word': 'Você'}, {'tag': '...",new-session,Você já pensou no que vai fazer nas férias ?,Não
14182,Preposição+Artigo,nas,interaction,4,oYoPEopG2Z,Preposição+Artigo,False,2025-05-30 16:52:53.776000+00:00,5-ensino-fundamental,"[{'tag': 'Pronome', 'word': 'Você'}, {'tag': '...",new-session,Você já pensou no que vai fazer nas férias ?,Não


In [34]:
model_inputs = []
targets = []
for sessionId in tqdm(df.sessionId.unique()):
    df_filtered = df[df.sessionId == sessionId].reset_index(drop=True)
    targets.append(df_filtered.loc[0, 'serie'])

    model_input = f"Frase original: \"{df_filtered.loc[0, 'sentence']}\"\nTentativa completa: {df_filtered.loc[0, 'complete_try']}. Interações do aluno:\n"
    for index, row in df_filtered.iterrows():
        model_input += (
            f"Palavra: {row['targetWord']}. "
            f"Tag correta: {row['correctTag']}. "
            f"Tag escolhida: {row['draggedTag']}. "
            f"Erro: {'sim' if row['isMistake'] else 'não'}. "
            f"Ordem da interação: {row['orderingId'] + 1}.\n"
        )

    model_input += "Fim."
    model_inputs.append(model_input)

dataset = pd.DataFrame()
dataset['input'] = model_inputs
dataset['target'] = targets

  0%|          | 0/895 [00:00<?, ?it/s]

In [35]:
dataset

,input,target
0,"Frase original: ""Será que alguém explicou o qu...",5-ensino-fundamental
1,"Frase original: ""Ufa , achei que não fosse che...",5-ensino-fundamental
2,"Frase original: ""A folha caiu da árvore .""\nTe...",6-ensino-fundamental
3,"Frase original: ""É importante que todos os alu...",6-ensino-fundamental
4,"Frase original: ""Não acredito que você tenha f...",9-ensino-fundamental
...,...,...
890,"Frase original: ""O cão mordeu o carteiro brinc...",8-ensino-fundamental
891,"Frase original: ""A menina caiu na escada e mac...",8-ensino-fundamental
892,"Frase original: ""O aluno nota 10 tirou uma not...",8-ensino-fundamental
893,"Frase original: ""Assim que terminou a reunião ...",9-ensino-fundamental


In [36]:
# Filtrando ensino médio (por enquanto)
dataset = dataset[(dataset.target != '1-ensino-medio')]
dataset = dataset[(dataset.target != '2-ensino-medio')]
dataset = dataset[(dataset.target != '3-ensino-medio')]
dataset = dataset.reset_index(drop=True)
dataset

,input,target
0,"Frase original: ""Será que alguém explicou o qu...",5-ensino-fundamental
1,"Frase original: ""Ufa , achei que não fosse che...",5-ensino-fundamental
2,"Frase original: ""A folha caiu da árvore .""\nTe...",6-ensino-fundamental
3,"Frase original: ""É importante que todos os alu...",6-ensino-fundamental
4,"Frase original: ""Não acredito que você tenha f...",9-ensino-fundamental
...,...,...
852,"Frase original: ""O cão mordeu o carteiro brinc...",8-ensino-fundamental
853,"Frase original: ""A menina caiu na escada e mac...",8-ensino-fundamental
854,"Frase original: ""O aluno nota 10 tirou uma not...",8-ensino-fundamental
855,"Frase original: ""Assim que terminou a reunião ...",9-ensino-fundamental


In [37]:
le = preprocessing.LabelEncoder()
le.fit(dataset.target)
dataset['label'] = le.transform(dataset.target)
dataset

,input,target,label
0,"Frase original: ""Será que alguém explicou o qu...",5-ensino-fundamental,1
1,"Frase original: ""Ufa , achei que não fosse che...",5-ensino-fundamental,1
2,"Frase original: ""A folha caiu da árvore .""\nTe...",6-ensino-fundamental,2
3,"Frase original: ""É importante que todos os alu...",6-ensino-fundamental,2
4,"Frase original: ""Não acredito que você tenha f...",9-ensino-fundamental,5
...,...,...,...
852,"Frase original: ""O cão mordeu o carteiro brinc...",8-ensino-fundamental,4
853,"Frase original: ""A menina caiu na escada e mac...",8-ensino-fundamental,4
854,"Frase original: ""O aluno nota 10 tirou uma not...",8-ensino-fundamental,4
855,"Frase original: ""Assim que terminou a reunião ...",9-ensino-fundamental,5


In [38]:
dataset.to_pickle('dataset.pkl')
joblib.dump(le, 'label_encoder.joblib')

['label_encoder.joblib']

# Instanciando modelo

In [62]:
df = pd.read_pickle('dataset.pkl')
df

,input,target,label
0,"Frase original: ""Será que alguém explicou o qu...",5-ensino-fundamental,1
1,"Frase original: ""Ufa , achei que não fosse che...",5-ensino-fundamental,1
2,"Frase original: ""A folha caiu da árvore .""\nTe...",6-ensino-fundamental,2
3,"Frase original: ""É importante que todos os alu...",6-ensino-fundamental,2
4,"Frase original: ""Não acredito que você tenha f...",9-ensino-fundamental,5
...,...,...,...
852,"Frase original: ""O cão mordeu o carteiro brinc...",8-ensino-fundamental,4
853,"Frase original: ""A menina caiu na escada e mac...",8-ensino-fundamental,4
854,"Frase original: ""O aluno nota 10 tirou uma not...",8-ensino-fundamental,4
855,"Frase original: ""Assim que terminou a reunião ...",9-ensino-fundamental,5


In [63]:
df.target.value_counts()

7-ensino-fundamental    266
6-ensino-fundamental    233
8-ensino-fundamental    162
9-ensino-fundamental    158
4-ensino-fundamental     21
5-ensino-fundamental     17
Name: target, dtype: int64

In [64]:
df.label.value_counts()

3    266
2    233
4    162
5    158
0     21
1     17
Name: label, dtype: int64

In [65]:
df.label.value_counts(True)

3    0.310385
2    0.271879
4    0.189032
5    0.184364
0    0.024504
1    0.019837
Name: label, dtype: float64

In [66]:
dataset = Dataset.from_pandas(df[['input', 'label']])
dataset = dataset.train_test_split(test_size=0.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'label'],
        num_rows: 685
    })
    test: Dataset({
        features: ['input', 'label'],
        num_rows: 172
    })
})

In [67]:
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

def tokenize(example):
    return tokenizer(example['input'], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/685 [00:00<?, ? examples/s]

Map:   0%|          | 0/172 [00:00<?, ? examples/s]

In [68]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 685
    })
    test: Dataset({
        features: ['input', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 172
    })
})

In [69]:
joblib.dump(tokenized_dataset, "tokenized_dataset.joblib")

['tokenized_dataset.joblib']

In [70]:
num_labels = df['label'].nunique()

model = AutoModelForSequenceClassification.from_pretrained(
    "neuralmind/bert-base-portuguese-cased",
    num_labels=num_labels
)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

In [ ]:
import evaluate
import torch
from transformers import TrainerCallback, TrainingArguments, Trainer

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), axis=-1)
    return accuracy.compute(predictions=preds, references=labels)

class EvalAndSaveBestModelCallback(TrainerCallback):
    def __init__(self, save_path):
        self.save_path = save_path
        self.best_accuracy = 0.0

    def on_epoch_end(self, args, state, control, **kwargs):
        print(f"\n→ End of epoch {int(state.epoch)}, evaluating...")
        metrics = self.trainer.evaluate()
        acc = metrics.get("eval_accuracy", 0.0)
        print(f"Evaluation Accuracy: {acc:.4f}")
        
        if acc > self.best_accuracy:
            print(f"→ New best accuracy! Saving model to {self.save_path}")
            self.best_accuracy = acc
            self.trainer.save_model(self.save_path)
            self.trainer.tokenizer.save_pretrained(self.save_path)

save_best_model_path = "./results/best_model"

eval_callback = EvalAndSaveBestModelCallback(save_path=save_best_model_path)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    logging_dir="./logs",
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[eval_callback],
)

eval_callback.trainer = trainer

trainer.train()


/home/daniel/anaconda3/envs/symbolic_transformer/lib/python3.7/site-packages/transformers/optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/daniel/anaconda3/envs/symbolic_transformer/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Accuracy
22,No log,1.538400,0.308140
44,No log,1.537581,0.284884
66,No log,1.515856,0.325581
88,No log,1.504658,0.313953
110,No log,1.482500,0.302326
132,No log,1.469839,0.366279
154,No log,1.487197,0.354651
176,No log,1.528389,0.308140
198,No log,1.560963,0.319767
220,No log,1.552545,0.325581



→ End of epoch 1, evaluating...
Evaluation Accuracy: 0.3081
→ New best accuracy! Saving model to ./results/best_model


/home/daniel/anaconda3/envs/symbolic_transformer/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '



→ End of epoch 2, evaluating...
Evaluation Accuracy: 0.2849


...

/home/daniel/anaconda3/envs/symbolic_transformer/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '



→ End of epoch 19, evaluating...
Evaluation Accuracy: 0.2791


/home/daniel/anaconda3/envs/symbolic_transformer/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '



→ End of epoch 20, evaluating...
Evaluation Accuracy: 0.2733


TrainOutput(global_step=440, training_loss=1.2711318969726562, metrics={'train_runtime': 66.0189, 'train_samples_per_second': 207.516, 'train_steps_per_second': 6.665, 'total_flos': 901187729049600.0, 'train_loss': 1.2711318969726562, 'epoch': 20.0})

In [45]:
model = AutoModelForSequenceClassification.from_pretrained("./results/best_model")
tokenizer = AutoTokenizer.from_pretrained("./results/best_model")

# Evaluating

In [4]:
tokenized_dataset = joblib.load("tokenized_dataset.joblib")

In [5]:
model_path = "./results/best_model"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [6]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), axis=-1)
    return accuracy.compute(predictions=preds, references=labels)

In [7]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results/eval",
    per_device_eval_batch_size=16,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Run evaluation on test set
metrics = trainer.evaluate(eval_dataset=tokenized_dataset["test"])
print(metrics)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/daniel/anaconda3/envs/symbolic_transformer/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 1.4698394536972046, 'eval_accuracy': 0.36627906976744184, 'eval_runtime': 1.5519, 'eval_samples_per_second': 110.831, 'eval_steps_per_second': 3.866}
